In [1]:
from omegaconf import OmegaConf
import os
import panel as pn
import numpy as np
import pyvista as pv

from torch_points3d.datasets.segmentation.ept_dataset import EptDataset
pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'


In [2]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.load(os.path.join(DIR,'../conf/data/segmentation/ept/ground/ground.yaml'))

In [3]:
dataset_options.data.dataroot = "/mnt/c/data/potreeconvert"
dataset = EptDataset(dataset_options.data)
print(dataset)

Dataset: EptDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = None
test_transform = Compose([
    SaveOriginalPosId,
    XYZFeature(axis=['x', 'y', 'z']),
    AddFeatsByKeys(pos_z=True, pos_x=True, pos_y=True),
    GridSampling3D(grid_size=0.5, quantize_coords=True, mode=last),
])
train_transform = Compose([
    XYZFeature(axis=['x', 'y', 'z']),
    AddFeatsByKeys(pos_z=True, pos_x=True, pos_y=True),
    GridSampling3D(grid_size=0.5, quantize_coords=True, mode=last),
])
val_transform = Compose([
    SaveOriginalPosId,
    XYZFeature(axis=['x', 'y', 'z']),
    AddFeatsByKeys(pos_z=True, pos_x=True, pos_y=True),
    GridSampling3D(grid_size=0.5, quantize_coords=True, mode=last),
])
inference_transform = Compose([
    SaveOriginalPosId,
    XYZFeature(axis=['x', 'y', 'z']),
    AddFeatsByKeys(pos_z=True, pos_x=True, pos_y=True),
    GridSampling3D(grid_size=0.5, quantize_coords=True, mode=last),

In [4]:
x = dataset.train_dataset[0]

(43547, 3)
(45400, 3)
(45811, 3)
(45908, 3)
(45933, 3)
(45941, 3)
(45943, 3)


In [5]:
x.y.max()

1

In [6]:
x.pos.min(axis=0)

torch.return_types.min(
values=tensor([-46.8485, -48.8215,  -3.0032]),
indices=tensor([ 2469, 22713,     5]))

In [7]:
x.pos.max(axis=0)

torch.return_types.max(
values=tensor([46.8495, 48.8215,  3.2638]),
indices=tensor([ 3097, 13243, 26240]))

In [8]:
x.pos.mean(axis=0)

tensor([-12.4103,  -5.9556,  -0.0400])

In [9]:
dataset.train_dataset.files[0]

DataFile(files=['/mnt/c/data/potreeconvert/ahn/ept-data/6-33-28-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/5-16-14-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/4-8-7-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/3-4-3-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/2-2-1-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/1-1-0-0.laz', '/mnt/c/data/potreeconvert/ahn/ept-data/0-0-0-0.laz'], mins=[133222.65573437497, 546484.3745624999, -6.364], maxs=[133320.31196874997, 546582.0307968749, 91.29223437499925], mid=array([1.33271656e+05, 5.46533375e+05, 4.26360000e+01]), dset_idx=0)

In [10]:
(x.pos.max() - x.pos.min()) / 2

tensor(48.8215)

In [11]:
len(x.pos)

26242

In [14]:
from joblib import Parallel, delayed
import json
from tqdm import tqdm
import time
import laspy

in_dir = "/mnt/c/data/potreeconvert/ahn"

heirarchyPath = os.path.join(in_dir, "ept-hierarchy", "0-0-0-0.json")

print("[powerlineProcess] reading ept file %s: " % (heirarchyPath))
start_time = time.time()

with open(heirarchyPath) as f:
    hierarchy = json.load(f)
filelist = [os.path.join(in_dir, "ept-data", x + ".laz") for x in hierarchy.keys()]

def _read_las(file):
    las = laspy.read(file)            
    classes = np.copy(las.classification)
    xyz = np.stack([las.x, las.y, las.z], axis=1)
    return xyz, classes

xyzs, classes = Parallel(n_jobs=-1)(delayed(_read_las)(file) for file in tqdm(filelist))



  0%|          | 0/13525 [00:00<?, ?it/s]

[powerlineProcess] reading ept file /mnt/c/data/potreeconvert/ahn/ept-hierarchy/0-0-0-0.json: 



 35%|███▌      | 4776/13525 [00:29<00:49, 178.17it/s]

KeyboardInterrupt: 


 35%|███▌      | 4776/13525 [00:42<00:49, 178.17it/s]